In [58]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [59]:
load_dotenv()

True

In [60]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [61]:
class JokeState(TypedDict):
    topic:str
    joke:str
    explanation:str


In [62]:
def generate(state:JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content
    return {'joke':response}

In [63]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [64]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke',generate)
graph.add_node('generate_explanation',generate_explanation)

graph.add_edge(START,'generate_joke')
graph.add_edge('generate_joke','generate_explanation')
graph.add_edge('generate_explanation',END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [65]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough!',
 'explanation': 'The joke relies on a **pun**, which is a play on words that uses a word with two different meanings.\n\nHere\'s the breakdown:\n\n*   **Dough** has two meanings:\n    *   The raw mixture of flour, water, and other ingredients used to make bread, pizza crust, etc. (the literal meaning in the context of a pizza maker)\n    *   Slang for **money** (the intended meaning to make the joke funny)\n\nThe joke suggests the pizza maker quit because he was tired of handling the pizza dough (the literal meaning). However, the humorous interpretation is that he was tired of receiving money (the slang meaning), implying he had enough money and didn\'t need the job anymore.\n\nThe humor comes from the unexpected twist in meaning when you realize "dough" is being used as a slang term for money.'}

In [66]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough!', 'explanation': 'The joke relies on a **pun**, which is a play on words that uses a word with two different meanings.\n\nHere\'s the breakdown:\n\n*   **Dough** has two meanings:\n    *   The raw mixture of flour, water, and other ingredients used to make bread, pizza crust, etc. (the literal meaning in the context of a pizza maker)\n    *   Slang for **money** (the intended meaning to make the joke funny)\n\nThe joke suggests the pizza maker quit because he was tired of handling the pizza dough (the literal meaning). However, the humorous interpretation is that he was tired of receiving money (the slang meaning), implying he had enough money and didn\'t need the job anymore.\n\nThe humor comes from the unexpected twist in meaning when you realize "dough" is being used as a slang term for money.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpo

In [67]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough!', 'explanation': 'The joke relies on a **pun**, which is a play on words that uses a word with two different meanings.\n\nHere\'s the breakdown:\n\n*   **Dough** has two meanings:\n    *   The raw mixture of flour, water, and other ingredients used to make bread, pizza crust, etc. (the literal meaning in the context of a pizza maker)\n    *   Slang for **money** (the intended meaning to make the joke funny)\n\nThe joke suggests the pizza maker quit because he was tired of handling the pizza dough (the literal meaning). However, the humorous interpretation is that he was tired of receiving money (the slang meaning), implying he had enough money and didn\'t need the job anymore.\n\nThe humor comes from the unexpected twist in meaning when you realize "dough" is being used as a slang term for money.'}, next=(), config={'configurable': {'thread_id': '1', 'checkp

In [68]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f084995-c3e1-6632-8000-4e5b798fe86d"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f084995-c3e1-6632-8000-4e5b798fe86d'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-29T05:31:06.092088+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f084995-c3de-6f1a-bfff-9f7798c020bd'}}, tasks=(PregelTask(id='6071cf41-df9b-46d7-7c19-5a6b8150d05f', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough!'}),), interrupts=())

In [69]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f084995-c3e1-6632-8000-4e5b798fe86d"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!',
 'explanation': 'The joke plays on the word "dominated" by replacing it with "dough-minated." Here\'s the breakdown:\n\n* **Dominated:** This word means being controlled or ruled by someone or something else. A job can be "dominating" if it\'s stressful, demanding, and leaves you feeling controlled.\n\n* **Dough:** This is the main ingredient in pizza crust.\n\n* **Dough-minated:** This is a pun combining "dough" and "dominated." The pizza maker is literally dealing with dough all day.  The joke implies that he\'s tired of being controlled or overwhelmed by the constant need to work with dough, or perhaps by the demands of making pizza.\n\nThe humor comes from the unexpected substitution of "dominated" with a word related to the context of the joke (pizza making), creating a silly and relatable reason for quitting a job. It\'s a lighthearted and punny way to express jo

In [70]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!', 'explanation': 'The joke plays on the word "dominated" by replacing it with "dough-minated." Here\'s the breakdown:\n\n* **Dominated:** This word means being controlled or ruled by someone or something else. A job can be "dominating" if it\'s stressful, demanding, and leaves you feeling controlled.\n\n* **Dough:** This is the main ingredient in pizza crust.\n\n* **Dough-minated:** This is a pun combining "dough" and "dominated." The pizza maker is literally dealing with dough all day.  The joke implies that he\'s tired of being controlled or overwhelmed by the constant need to work with dough, or perhaps by the demands of making pizza.\n\nThe humor comes from the unexpected substitution of "dominated" with a word related to the context of the joke (pizza making), creating a silly and relatable reason for quitting a job. It\'s a lighthearted and pun

In [71]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f084995-c3e1-6632-8000-4e5b798fe86d", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f08499c-5a2f-63b4-8001-71a9a3ade13e'}}

In [72]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08499c-5a2f-63b4-8001-71a9a3ade13e'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-08-29T05:34:02.913886+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f084995-c3e1-6632-8000-4e5b798fe86d'}}, tasks=(PregelTask(id='98067c37-2e3f-f00b-2c44-0f5128aece1d', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!', 'explanation': 'The joke plays on the word "dominated" by replacing it with "dough-minated." Here\'s the breakdown:\n\n* **Dominated:** This word means being controlled or ruled by someone or something else. A job can be "dominating" if it\

In [73]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti blush?\n\nBecause it saw the meat sauce!',
 'explanation': 'The joke relies on a double entendre, playing on the word "sauce" and the idea of blushing. Here\'s the breakdown:\n\n* **Literal Meaning:** Spaghetti is a type of pasta, and meat sauce is a common topping for it.  The joke presents a scenario where the spaghetti "blushes" when it sees the meat sauce.\n\n* **Figurative/Sexual Meaning:**  "Sauce" can also refer to something that adds spice or flavor to life, often with a sexual connotation.  "Blushing" is a common reaction to embarrassment or arousal. The joke implies that the spaghetti is blushing because the meat sauce is seen as sexually suggestive or exciting.\n\n**In essence, the joke is funny because it personifies the spaghetti and uses the double meaning of "sauce" to create a slightly suggestive and humorous scenario. The unexpected blush adds to the comedic effect.**'}

In [74]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti blush?\n\nBecause it saw the meat sauce!', 'explanation': 'The joke relies on a double entendre, playing on the word "sauce" and the idea of blushing. Here\'s the breakdown:\n\n* **Literal Meaning:** Spaghetti is a type of pasta, and meat sauce is a common topping for it.  The joke presents a scenario where the spaghetti "blushes" when it sees the meat sauce.\n\n* **Figurative/Sexual Meaning:**  "Sauce" can also refer to something that adds spice or flavor to life, often with a sexual connotation.  "Blushing" is a common reaction to embarrassment or arousal. The joke implies that the spaghetti is blushing because the meat sauce is seen as sexually suggestive or exciting.\n\n**In essence, the joke is funny because it personifies the spaghetti and uses the double meaning of "sauce" to create a slightly suggestive and humorous scenario. The unexpected blush adds to the comedic effect.**'}, next=(), config={'configurab

In [39]:
#FaultTolerence

In [40]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [41]:
class CrashState(TypedDict):
    input:str
    step1:str
    step2:str


In [42]:
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [43]:
graph = StateGraph(CrashState)

graph.add_node('step1',step_1)
graph.add_node('step2',step_2)
graph.add_node('step3',step_3)

graph.add_edge(START,'step1')
graph.add_edge('step1','step2')
graph.add_edge('step2','step3')
graph.add_edge('step3',END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [44]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...


AttributeError: 'StateGraph' object has no attribute 'invoke'

In [ ]:

# First run, stop *after step2*
events = workflow.stream(
    {"input": "start"},
    config={
        "configurable": {"thread_id": "thread-1"}
    },
    interrupt_after=["step2"],  # 🚨 interrupt point
)

for event in events:
    print(event)   # You’ll see events until step2 finishes


✅ Step 1 executed
{'step1': {'step1': 'done', 'input': 'start'}}
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
{'step2': {'step2': 'done'}}
{'__interrupt__': ()}


In [ ]:
list(workflow.get_state_history(config={
        "configurable": {"thread_id": "thread-1"}
    }))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08493b-332c-6913-8002-73ac45891178'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-29T04:50:34.999425+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08493b-332a-6202-8001-a90656c1680d'}}, tasks=(PregelTask(id='0b9fbf9e-f64b-7f26-1353-316f739f1a01', name='step3', path=('__pregel_pull', 'step3'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08493b-332a-6202-8001-a90656c1680d'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-29T04:50:34.998425+00:00', parent_config={'configurable': {'thread_id': 'threa

In [ ]:
events = workflow.stream(
    None,   # no new input needed
    config={
        "configurable": {"thread_id": "thread-1"}
    }
)

for event in events:
    print(event)   # Now runs step3 → END


✅ Step 3 executed
{'step3': None}


In [ ]:
list(workflow.get_state_history(config={
        "configurable": {"thread_id": "thread-1"}
    }))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f084940-5585-6f54-8003-174053cf1589'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-08-29T04:52:52.818926+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08493b-332c-6913-8002-73ac45891178'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08493b-332c-6913-8002-73ac45891178'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-29T04:50:34.999425+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08493b-332a-6202-8001-a90656c1680d'}}, tasks=(PregelTask(id='0b9fbf9e-f64b-7f26-1353-316f739f1a01